# QC for Value Sets and Enums

This tutorial shows how to check data quality for LinkML value sets (enums with permissible values).

## The Challenge

Value sets often have permissible values that should include:
- `description` - Human-readable explanation
- `meaning` - Mapping to an ontology term (CURIE)
- `title` - Display label

However, the LinkML metamodel doesn't mark these as `recommended: true`, so we need to:
1. Extract permissible values from the schema
2. Define our own QC schema with recommended fields
3. Analyze the PVs as data

## Setup: Create a QC Schema for Permissible Values

In [1]:
%%bash
cat > /tmp/pv_qc_schema.yaml << 'EOF'
id: https://example.org/pv-qc
name: permissible_value_qc
description: Schema for QC of permissible values in value sets
prefixes:
  linkml: https://w3id.org/linkml/
imports:
  - linkml:types
default_range: string

classes:
  PermissibleValue:
    description: A permissible value in an enumeration
    attributes:
      text:
        description: The text/code for this value
        required: true
        identifier: true
      title:
        description: Human-readable display label
        recommended: true
      description:
        description: Detailed explanation of this value
        recommended: true
      meaning:
        description: Ontology term CURIE mapping
        recommended: true
      # Additional fields we might want to track
      unit:
        recommended: true
EOF
echo "PV QC schema created!"

PV QC schema created!


## Extract Permissible Values from a Value Set

Let's create a sample value set and extract its PVs:

In [2]:
%%bash
# Create a sample value set (like those in the valuesets repo)
cat > /tmp/sample_valueset.yaml << 'EOF'
id: https://example.org/valuesets/SampleStatus
name: SampleStatus
prefixes:
  linkml: https://w3id.org/linkml/
  OBI: http://purl.obolibrary.org/obo/OBI_
  EFO: http://www.ebi.ac.uk/efo/EFO_
imports:
  - linkml:types
enums:
  SampleStatusEnum:
    description: Status of a biological sample
    permissible_values:
      COLLECTED:
        text: COLLECTED
        title: Sample Collected
        description: Sample has been collected from the source
        meaning: OBI:0000659
      PROCESSED:
        text: PROCESSED
        title: Sample Processed  
        description: Sample has undergone processing
        meaning: OBI:0000094
      STORED:
        text: STORED
        title: In Storage
        description: Sample is in long-term storage
        meaning: OBI:0002119
      DEPLETED:
        text: DEPLETED
        title: Sample Depleted
        # Missing description!
        meaning: EFO:0009089
      CONTAMINATED:
        text: CONTAMINATED
        title: Contaminated
        description: Sample has been contaminated
        # Missing meaning!
      LOST:
        text: LOST
        # Missing title, description, and meaning!
      UNKNOWN:
        text: UNKNOWN
        title: Unknown Status
        # Missing description and meaning
EOF
echo "Sample valueset created!"

Sample valueset created!


## Extract PVs and Convert to Data Format

We'll use Python to extract permissible values and write them as data:

In [3]:
import yaml


def extract_pvs_from_schema(schema_path: str) -> list[dict]:
    """Extract all permissible values from a LinkML schema."""
    with open(schema_path) as f:
        schema = yaml.safe_load(f)

    pvs = []
    for enum_name, enum_def in schema.get('enums', {}).items():
        for pv_name, pv_def in enum_def.get('permissible_values', {}).items():
            pv = {'text': pv_name}
            if isinstance(pv_def, dict):
                pv.update(pv_def)
            pvs.append(pv)

    return pvs


# Extract PVs
pvs = extract_pvs_from_schema('/tmp/sample_valueset.yaml')
print(f"Extracted {len(pvs)} permissible values:")
for pv in pvs:
    print(f"  {pv.get('text')}: meaning={pv.get('meaning', 'MISSING')}")

Extracted 7 permissible values:
  COLLECTED: meaning=OBI:0000659
  PROCESSED: meaning=OBI:0000094
  STORED: meaning=OBI:0002119
  DEPLETED: meaning=EFO:0009089
  CONTAMINATED: meaning=MISSING
  LOST: meaning=MISSING
  UNKNOWN: meaning=MISSING


In [4]:
# Write PVs as YAML data for QC
with open('/tmp/pvs_data.yaml', 'w') as f:
    yaml.dump(pvs, f)

print("PVs written to /tmp/pvs_data.yaml")
print("\nContents:")
with open('/tmp/pvs_data.yaml') as f:
    print(f.read())

PVs written to /tmp/pvs_data.yaml

Contents:
- description: Sample has been collected from the source
  meaning: OBI:0000659
  text: COLLECTED
  title: Sample Collected
- description: Sample has undergone processing
  meaning: OBI:0000094
  text: PROCESSED
  title: Sample Processed
- description: Sample is in long-term storage
  meaning: OBI:0002119
  text: STORED
  title: In Storage
- meaning: EFO:0009089
  text: DEPLETED
  title: Sample Depleted
- description: Sample has been contaminated
  text: CONTAMINATED
  title: Contaminated
- text: LOST
- text: UNKNOWN
  title: Unknown Status



## Run QC on Permissible Values

In [5]:
%%bash
linkml-data-qc /tmp/pvs_data.yaml \
    -s /tmp/pv_qc_schema.yaml \
    -t PermissibleValue

Compliance Report: /tmp/pvs_data.yaml
Target Class: PermissibleValue
Global Compliance: 50.0% (14/28

)
Weighted Compliance: 50.0%

Summary by Slot:
  description: 57.1%
  meaning: 57.1%
  title: 85.7%


  unit: 0.0%

Aggregated Scores by List Path:
  [].description: 57.1% (4/7)
  [].meaning: 57.1% (4/7

)
  [].title: 85.7% (6/7)
  [].unit: 0.0% (0/7)

Detailed Path Scores:
  [0] (PermissibleValue): 75.

0%
    - title: OK
    - description: OK
    - meaning: OK
    - unit: MISSING
  [1] (PermissibleVal

...

 (PermissibleValue): 50.0%
    - title: OK
    - description: MISSING
    - meaning: OK
    - unit: 

MISSING
  [4] (PermissibleValue): 50.0%
    - title: OK
    - description: OK
    - meaning: MISSING


    - unit: MISSING
  [5] (PermissibleValue): 0.0%
    - title: MISSING
    - description: MISSING


    - meaning: MISSING
    - unit: MISSING
  [6] (PermissibleValue): 25.0%
    - title: OK
    - des

cription: MISSING
    - meaning: MISSING
    - unit: MISSING


## Configure Weights and Thresholds

We can prioritize `meaning` (ontology mappings) as most important:

In [6]:
%%bash
cat > /tmp/pv_qc_config.yaml << 'EOF'
default_weight: 1.0

slots:
  meaning:
    weight: 3.0           # Ontology mappings are critical
    min_compliance: 80.0  # At least 80% should have mappings
  description:
    weight: 2.0           # Descriptions are important
    min_compliance: 70.0
  title:
    weight: 1.5           # Titles are nice to have
    min_compliance: 90.0
EOF

linkml-data-qc /tmp/pvs_data.yaml \
    -s /tmp/pv_qc_schema.yaml \
    -t PermissibleValue \
    -c /tmp/pv_qc_config.yaml

Compliance Report: /tmp/pvs_data.yaml
Target Class: PermissibleValue
Global Compliance: 50.0% (14/28

)
Weighted Compliance: 55.2%
Config: /tmp/pv_qc_config.yaml

Summary by Slot:
  description: 57.1%
 

 meaning: 57.1%
  title: 85.7%
  unit: 0.0%

Threshold Violations (3):
  [].description: 57.1% < 70.

0% (shortfall: 12.9%)
  [].meaning: 57.1% < 80.0% (shortfall: 22.9%)
  [].title: 85.7% < 90.0% (shor

tfall: 4.3%)

Aggregated Scores by List Path:
  [].description: 57.1% (4/7)
  [].meaning: 57.1% (4/7

...

 (PermissibleValue): 50.0%
    - title: OK
    - description: MISSING
    - meaning: OK
    - unit: 

MISSING
  [4] (PermissibleValue): 50.0%
    - title: OK
    - description: OK
    - meaning: MISSING


    - unit: MISSING
  [5] (PermissibleValue): 0.0%
    - title: MISSING
    - description: MISSING


    - meaning: MISSING
    - unit: MISSING
  [6] (PermissibleValue): 25.0%
    - title: OK
    - des

cription: MISSING
    - meaning: MISSING
    - unit: MISSING


## Analyzing Multiple Value Sets

Let's create a function to QC an entire directory of value sets:

In [7]:
from pathlib import Path

from linkml_data_qc import ComplianceAnalyzer, QCConfig, SlotQCConfig


def analyze_valueset_directory(vs_dir: str, schema_path: str, config: QCConfig = None):
    """Analyze all value sets in a directory."""
    results = []

    for vs_file in Path(vs_dir).glob('*.yaml'):
        try:
            pvs = extract_pvs_from_schema(str(vs_file))
            if not pvs:
                continue

            # Write to temp file
            temp_path = f'/tmp/pvs_{vs_file.stem}.yaml'
            with open(temp_path, 'w') as f:
                yaml.dump(pvs, f)

            # Analyze
            analyzer = ComplianceAnalyzer(schema_path, config)
            report = analyzer.analyze_file(temp_path, 'PermissibleValue')

            results.append({
                'file': vs_file.name,
                'pv_count': len(pvs),
                'compliance': report.global_compliance,
                'meaning_pct': report.summary_by_slot.get('meaning', 0),
                'description_pct': report.summary_by_slot.get('description', 0),
            })
        except Exception as e:
            print(f"Error processing {vs_file}: {e}")

    return results


# Create another sample valueset
with open('/tmp/sample_valueset2.yaml', 'w') as f:
    yaml.dump({
        'id': 'https://example.org/vs2',
        'name': 'test2',
        'enums': {
            'TestEnum': {
                'permissible_values': {
                    'A': {'text': 'A', 'title': 'Value A', 'description': 'First value', 'meaning': 'EX:001'},
                    'B': {'text': 'B', 'title': 'Value B', 'description': 'Second value', 'meaning': 'EX:002'},
                }
            }
        }
    }, f)

# Analyze
config = QCConfig(
    slots={
        'meaning': SlotQCConfig(weight=3.0),
        'description': SlotQCConfig(weight=2.0),
    }
)

results = analyze_valueset_directory('/tmp', '/tmp/pv_qc_schema.yaml', config)

print("\n=== Value Set QC Summary ===")
print(f"{'File':<30} {'PVs':>5} {'Overall':>8} {'Meaning':>8} {'Desc':>8}")
print("-" * 65)
for r in sorted(results, key=lambda x: x['compliance']):
    print(f"{r['file']:<30} {r['pv_count']:>5} {r['compliance']:>7.1f}% {r['meaning_pct']:>7.1f}% {r['description_pct']:>7.1f}%")

Error processing /tmp/pvs_data.yaml: 'list' object has no attribute 'get'


Error processing /tmp/celltype_pvs.yaml: 'list' object has no attribute 'get'
Error processing /tmp/pvs_sample_valueset2.yaml: 'list' object has no attribute 'get'
Error processing /tmp/pvs_sample_valueset.yaml: 'list' object has no attribute 'get'

=== Value Set QC Summary ===
File                             PVs  Overall  Meaning     Desc
-----------------------------------------------------------------
sample_valueset.yaml               7    50.0%    57.1%    57.1%
sample_valueset2.yaml              2    75.0%   100.0%   100.0%


## Real-World Example: Cell Type Value Set

Let's analyze a real value set from the valuesets repository (if available):

In [8]:
# Try to find a real valueset
vs_path = Path('/Users/cjm/repos/valuesets/src/valuesets/expanded/bio/CellType.yaml')

if vs_path.exists():
    pvs = extract_pvs_from_schema(str(vs_path))
    print(f"Found {len(pvs)} permissible values in CellType")

    # Sample some PVs
    print("\nSample PVs:")
    for pv in pvs[:5]:
        has_meaning = '✓' if pv.get('meaning') else '✗'
        has_desc = '✓' if pv.get('description') else '✗'
        print(f"  {pv.get('text', 'N/A')}: meaning={has_meaning} desc={has_desc}")

    # Write and analyze
    with open('/tmp/celltype_pvs.yaml', 'w') as f:
        yaml.dump(pvs, f)
else:
    print("CellType valueset not found - using sample data")

Found 3119 permissible values in CellType

Sample PVs:
  CL:0011105: meaning=✓ desc=✓
  CL:0002656: meaning=✓ desc=✓
  CL:4033014: meaning=✓ desc=✓
  CL:4030031: meaning=✓ desc=✓
  CL:4052036: meaning=✓ desc=✓


In [9]:
%%bash
if [ -f /tmp/celltype_pvs.yaml ]; then
    echo "=== CellType Value Set QC ==="
    linkml-data-qc /tmp/celltype_pvs.yaml \
        -s /tmp/pv_qc_schema.yaml \
        -t PermissibleValue \
        -c /tmp/pv_qc_config.yaml
else
    echo "No CellType data available"
fi

=== CellType Value Set QC ===


Compliance Report: /tmp/celltype_pvs.yaml
Target Class: PermissibleValue
Global Compliance: 74.2% (9

252/12476)
Weighted Compliance: 85.8%
Config: /tmp/pv_qc_config.yaml

Summary by Slot:
  description

: 96.6%
  meaning: 100.0%
  title: 100.0%
  unit: 0.0%

Aggregated Scores by List Path:
  [].descrip

tion: 96.6% (3014/3119)
  [].meaning: 100.0% (3119/3119)
  [].title: 100.0% (3119/3119)
  [].unit: 0

...

g: OK
    - unit: MISSING
  [3115] (PermissibleValue): 50.0%
    - title: OK
    - description: MISS

ING
    - meaning: OK
    - unit: MISSING
  [3116] (PermissibleValue): 50.0%
    - title: OK
    - d

escription: MISSING
    - meaning: OK
    - unit: MISSING
  [3117] (PermissibleValue): 75.0%
    - t

itle: OK
    - description: OK
    - meaning: OK
    - unit: MISSING
  [3118] (PermissibleValue): 75

.0%
    - title: OK
    - description: OK
    - meaning: OK
    - unit: MISSING


## Key Takeaways

1. **Schema-as-data**: Treat LinkML schemas as data conforming to the metamodel
2. **Custom QC schema**: Define your own schema with `recommended: true` for fields that matter
3. **Extraction**: Write a simple function to extract PVs from enum definitions
4. **Configuration**: Use weights to prioritize `meaning` (ontology mappings) over descriptions
5. **Automation**: Wrap in a function to process entire directories of value sets

## CI/CD Integration

```bash
# Extract PVs from all schemas
python extract_pvs.py schemas/*.yaml > pvs.yaml

# Run QC with 80% meaning coverage requirement
linkml-data-qc pvs.yaml \
    -s pv_qc_schema.yaml \
    -t PermissibleValue \
    -c pv_config.yaml \
    --fail-on-violations
```